# Pruebas declaraciones

In [1]:
import pandas as pd
import configparser
import openai
from pydantic import BaseModel
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
import json

In [2]:
data = pd.read_csv("../data/contenido_2023_04_05_matriz_not_julia.csv")
data

,Unnamed: 0,N,n_periodico,enlace,Medio,año,Fecha,Nombreperiodista,Nombreperiodista_A,Género_periodista,...,@25ge3ero1oto,@25perspectiva,@25piedefoto,@26ge3ero1oto,@26perspectiva,@26piedefoto,@27ge3ero1oto,@27perspectiva,@27piedefoto,contenido
0,0,12,12,https://www.infolibre.es/videolibre/como-lo-ve...,1,2021,10/10/2021,Antonio Contreras,ANTONIO CONTRERAS,0,...,,,,,,,,,,"Antonio Lucas (Madrid, 1975) acaba de publicar..."
1,1,438,276,https://www.lavanguardia.com/participacion/car...,2,2021,28/1/2021,ANDREU YAKUBUV-TREMBACH,ANDREU YAKUBUV-TREMBACH,0,...,,,,,,,,,,Distinguir entre las esferas íntima y privada ...
2,2,1411,414,https://www.elmundo.es/economia/2020/04/08/5e8...,3,2020,8/4/2020,EDUARDO FERNÁNDEZ,EDUARDO FERNÁNDEZ,0,...,,,,,,,,,,Directo. Última hora sobre el coronavirus\n\nL...
3,3,1418,421,https://www.elmundo.es/vida-sana/familia-y-co/...,3,2020,20/3/2020,MAR MUÑIZ,MAR MUÑIZ,1,...,,,,,,,,,,Coronavirus: ¿nos convalidarán Teleco por tele...
4,4,1836,839,https://www.elmundo.es/papel/historias/2019/03...,3,2019,3/3/2019,FERMÍN ELIZARI,FERMÍN ELIZARI,0,...,,,,,,,,,,15.000 personas de todo el mundo revisan las p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122,4130,4130,1325,https://elpais.com/tecnologia/2017/01/17/actua...,4,2017,18/1/2017,MARYA G. NIETO,MARYA G. NIETO,1,...,,,,,,,,,,Dos algoritmos clave en el desarrollo de los a...
4123,4131,4131,1326,https://elpais.com/elpais/2017/01/05/eps/14835...,4,2017,5/1/2017,JAVIER MARTÍN DEL BARRIO,JAVIER MARTÍN DEL BARRIO,0,...,,,,,,,,,,RANA EL KALIOUBY no es una estrella de la canc...
4124,4132,4132,1327,https://elpais.com/elpais/2017/01/18/eps/14846...,4,2017,18/1/2017,GUILLERMO ABRIL,GUILLERMO ABRIL,0,...,,,,,,,,,,"CUANDO ATERRIZÓ en España, en septiembre de 20..."
4125,4133,4134,1329,https://elpais.com/elpais/2017/01/15/opinion/1...,4,2017,16/1/2017,JOAQUÍN PRIETO,JOAQUÍN PRIETO,0,...,,,,,,,,,,Las últimas campañas electorales celebradas en...


In [ ]:
data = data[:100]
data

,Unnamed: 0,N,n_periodico,enlace,Medio,año,Fecha,Nombreperiodista,Nombreperiodista_A,Género_periodista,...,@25ge3ero1oto,@25perspectiva,@25piedefoto,@26ge3ero1oto,@26perspectiva,@26piedefoto,@27ge3ero1oto,@27perspectiva,@27piedefoto,contenido
0,0,12,12,https://www.infolibre.es/videolibre/como-lo-ve...,1,2021,10/10/2021,Antonio Contreras,ANTONIO CONTRERAS,0,...,,,,,,,,,,"Antonio Lucas (Madrid, 1975) acaba de publicar..."
1,1,438,276,https://www.lavanguardia.com/participacion/car...,2,2021,28/1/2021,ANDREU YAKUBUV-TREMBACH,ANDREU YAKUBUV-TREMBACH,0,...,,,,,,,,,,Distinguir entre las esferas íntima y privada ...
2,2,1411,414,https://www.elmundo.es/economia/2020/04/08/5e8...,3,2020,8/4/2020,EDUARDO FERNÁNDEZ,EDUARDO FERNÁNDEZ,0,...,,,,,,,,,,Directo. Última hora sobre el coronavirus\n\nL...
3,3,1418,421,https://www.elmundo.es/vida-sana/familia-y-co/...,3,2020,20/3/2020,MAR MUÑIZ,MAR MUÑIZ,1,...,,,,,,,,,,Coronavirus: ¿nos convalidarán Teleco por tele...
4,4,1836,839,https://www.elmundo.es/papel/historias/2019/03...,3,2019,3/3/2019,FERMÍN ELIZARI,FERMÍN ELIZARI,0,...,,,,,,,,,,15.000 personas de todo el mundo revisan las p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,4007,1202,https://elpais.com/ccaa/2017/04/21/catalunya/1...,4,2017,21/4/2017,CARLES GELI,CARLES GELI,0,...,,,,,,,,,,Hace un tiempo que los neurocientíficos buscan...
96,96,4037,1232,https://elpais.com/elpais/2017/03/30/icon/1490...,4,2017,30/3/2017,IÑIGO LÓPEZ PALACIOS,IÑIGO LÓPEZ PALACIOS,0,...,,,,,,,,,,Hay muchas cosas desconocidas del funcionamien...
97,97,4053,1248,https://elpais.com/tecnologia/2017/02/20/actua...,4,2017,22/2/2017,PATRICIA PEIRÓ,PATRICIA PEIRÓ,1,...,,,,,,,,,,Harriet Green dirige la división de Internet d...
98,98,4091,1286,https://elpais.com/tecnologia/2017/01/20/actua...,4,2017,22/1/2017,BEATRIZ GUILLÉN,BEATRIZ GUILLÉN,1,...,,,,,,,,,,"Regina Barzilay (1970, Chisináu, Moldavia) es ..."


In [ ]:
class Quote(BaseModel):
    cita_textual: str
    fuente: str
    genero_fuente: str
    colectivos: list[str]
    verbo: str

class ResponseFormat(BaseModel):
    quotes: list[Quote]
    num_quotes: int

config = configparser.ConfigParser()
config.read('config.ini')

OPENAI_MODEL = config['DEFAULT']['OPENAI_MODEL']
openai.api_key = config['DEFAULT']['OPENAI_API_KEY']

In [5]:
predictions = []

# Loop de predicción
for idx, row in tqdm(data.iterrows(), total=len(data), desc="Procesando predicciones"):
    n = row['N']
    n_periodico = row['n_periodico']
    contenido = row['contenido']
    try:
        # Llamada al modelo para detectar las citas humanas y sus fuentes
        completion = openai.beta.chat.completions.parse(
            model=OPENAI_MODEL,
            messages=[
                {
                    "role": "system", 
                    "content": "Eres un periodista especializado en la extracción de declaraciones de humanos en artículos de noticias. Tu tarea es identificar y extraer todas las declaraciones humanas del texto, incluyendo la cita textual (si está entre comillas), la fuente de la declaración (como nombre, cargo o afiliación) y el verbo o expresión utilizada para introducir la declaración. Debes ignorar cualquier declaración hecha por robots o entidades no humanas."
                },
                {
                    "role": "user", 
                    "content": f"""
                                Extrae todas las declaraciones humanas del siguiente artículo, centrándote en identificar las personas mencionadas y su género. Para cada declaración, proporciona:

                                - **Cita textual**: Si está presente en el texto.
                                - **Fuente**: La persona, cargo o afiliación que hizo la declaración.
                                - **Género de la fuente**: El género correspondiente de la persona que hizo la declaración (masculino, femenino, desconocido).
                                - **Verbo**: El verbo o expresión utilizada para introducir la declaración.
                                - **Colectivos en la declaración**: Colectivos a los que hace referencia la declaración.

                                Si el género no es explícito en el texto, márcalo como "desconocido".  

                                Ignora declaraciones hechas por robots o entidades no humanas.

                                Texto: {contenido}
                                """
                                                }
            ],
            response_format=ResponseFormat
        )
        
        # Extraer la respuesta
        response_message = completion.choices[0].message.content
        print(response_message)

        # Usamos json.loads para convertir el string JSON a un objeto Python
        response_data = json.loads(response_message)

        # Agregar el índice de la noticia al JSON
        response_data['indice'] = idx  # Agrega el índice al diccionario de cada cita
        response_data['N'] = n
        response_data['n_periodico'] = n_periodico
        
        # Agregar el resultado con el índice a las predicciones
        predictions.append(response_data)

    except Exception as e:
        print(f"Error en el índice {idx}: {e}")

Procesando predicciones:   1%|          | 1/100 [00:08<13:48,  8.37s/it]

{"quotes":[{"cita_textual":"\"Yo encontré en esos hombres el quilate más puro del humano: la nobleza, la lealtad, la bondad, una bondad sin orillas. La lección que yo me llevo de Gran Sol, curiosamente, no es el mar. Son los hombres que surcan un mar, que saben que su vida es aquello que ese mismo mar no les ha querido arrancar\"","fuente":"Antonio Lucas","genero_fuente":"masculino","colectivos":["hombres","marineros"] ,"verbo":"encontré"},{"cita_textual":"\"Al trabajar en un periódico, aunque yo siempre he pertenecido a la parte de cultura, te impregnas del día a día, de ese pulso, de esa vibración de la política.\"","fuente":"Antonio Lucas","genero_fuente":"masculino","colectivos":["día a día","política"],"verbo":"trabajar"},{"cita_textual":"\"Creo que uno de los momentos más importantes a nivel cívico de este país fue el 15M, donde, de repente, hubo una verdadera pulsión, o un apetito, por implicarse en la política, saber de política o estudiar política, o, por lo menos, discutir de

Procesando predicciones:   2%|▏         | 2/100 [00:17<14:00,  8.58s/it]

{"quotes":[{"cita_textual":"Aquélla es más amplia que ésta (…) constituye un conjunto más global de facetas de la personalidad que, aisladamente consideradas, pueden carecer de significación intrínseca, pero que, coherentemente enlazadas entre sí, arrojan como precipitado un retrato de la personalidad del individuo que éste tiene derecho a mantener reservado.","fuente":"Desconocido","genero_fuente":"desconocido","colectivos":[],"verbo":"cuenta"},{"cita_textual":"No hay derecho","fuente":"Desconocido","genero_fuente":"desconocido","colectivos":[],"verbo":"vivimos"},{"cita_textual":"El trato que se dispensa a la privacidad en regímenes como los de China y Rusia sigue siendo un reto del progreso.","fuente":"Desconocido","genero_fuente":"desconocido","colectivos":["habitantes de China","habitantes de Rusia"],"verbo":"advierte"},{"cita_textual":"la puesta en valor de la privacidad asienta lo que viene a ser nuestro modo de vida europeo.","fuente":"Desconocido","genero_fuente":"desconocido",

Procesando predicciones:   3%|▎         | 3/100 [00:18<08:24,  5.20s/it]

...

{"quotes":[{"cita_textual":"La sociedad ha generado más datos en dos años que en toda la historia","fuente":"Harriet Green","genero_fuente":"femenino","colectivos":["sociedad"] ,"verbo":"asegura"},{"cita_textual":"Los datos son el nuevo petróleo","fuente":"Harriet Green","genero_fuente":"femenino","colectivos":[],"verbo":"sentencia"},{"cita_textual":"Tenemos un punto de vista y es que ha dado comienzo la era cognitiva","fuente":"Harriet Green","genero_fuente":"femenino","colectivos":[],"verbo":"afirma"},{"cita_textual":"La capacidad de conectar cosas con cosas y a las personas con las cosas no la inventamos nosotros, nos estamos aprovechando de ella","fuente":"Harriet Green","genero_fuente":"femenino","colectivos":[],"verbo":"defiende"},{"cita_textual":"El temor es perfectamente entendible y sobre todo en las generaciones más jóvenes, pero sabemos los datos personales son el ADN de cada individuo y por eso los protegemos con los sistemas más avanzados de seguridad","fuente":"Harriet Gr

Procesando predicciones:  99%|█████████▉| 99/100 [16:12<00:08,  8.28s/it]

{"quotes":[{"cita_textual":"Intento que una máquina coja un texto, extraiga su información y lo haga útil para los humanos. Pero las máquinas tienen que aprender el lenguaje, porque ellas por sí mismas no saben leer inglés","fuente":"Regina Barzilay","genero_fuente":"femenino","colectivos":[],"verbo":"explica"},{"cita_textual":"Hoy en día tenemos mejor tecnología para recomendarte un pintalabios que para ayudarte a prevenir el cáncer de mama. Eso había que cambiarlo","fuente":"Regina Barzilay","genero_fuente":"femenino","colectivos":[],"verbo":""},{"cita_textual":"Porque me di cuenta de que en los hospitales hay muchísima información que se registra sobre los pacientes y que no se está utilizando. La mayoría de las decisiones clínicas están basadas en los ensayos clínicos, es decir, en el 3% de población que participan en ellos. Esto significa que toda la experiencia de lo que le pasa al 97% de pacientes no está siendo usada","fuente":"Regina Barzilay","genero_fuente":"femenino","colec

Procesando predicciones: 100%|██████████| 100/100 [16:22<00:00,  9.82s/it]

{"quotes":[{"cita_textual":"\"Imagino que la gente asume que con lo de Matrix me debe interesar esos temas. Pero no\"","fuente":"Carrie-Anne Moss","genero_fuente":"femenino","colectivos":["gente"] ,"verbo":"confiesa"},{"cita_textual":"\"Vinieron a mí con el personaje, lo atraje no sé bien por qué. Todavía estoy agradecida de que fuera así, porque incluso cuando vi la primera temporada quería ser parte de ella\"","fuente":"Carrie-Anne Moss","genero_fuente":"femenino","colectivos":[],"verbo":"cuenta"},{"cita_textual":"\"Son fantásticos. Me interesa cada uno de ellos y sus viajes personales\"","fuente":"Carrie-Anne Moss","genero_fuente":"femenino","colectivos":["personajes"],"verbo":"se queda"},{"cita_textual":"\"Me encantaría dar vida a uno de los robots. Me encanta cómo se mueven, los actores son tan buenos haciéndolo porque tienen mucho entrenamiento previo para lograrlo\"","fuente":"Carrie-Anne Moss","genero_fuente":"femenino","colectivos":["robots","actores"],"verbo":"confiesa"},{"ci

In [6]:
# Guardar las predicciones en un archivo JSON
with open('predictions.json', 'w', encoding='utf-8') as f:
    json.dump(predictions, f, ensure_ascii=False, indent=4)

In [7]:
def compute_metrics(true_labels, predicted_labels):

    # Obtener reporte completo
    report = classification_report(true_labels, predicted_labels, output_dict=True)
    
    # Obtener la matriz de confusión
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    
    # Extraer métricas para cada clase y globales
    metrics = {
        'accuracy': report['accuracy'],
        'weighted_f1': report['weighted avg']['f1-score'],
        'macro_f1': report['macro avg']['f1-score'],
        # 'weighted_precision': report['weighted avg']['precision'],
        # 'weighted_recall': report['weighted avg']['recall'],
        # La matriz de confusión no se incluye normalmente como una métrica devuelta porque no es un escalar
        'confusion_matrix': conf_matrix.tolist()  # Convertir a lista para asegurarse de que es serializable si es necesario
    }
    
    # # Añadir métricas específicas por clase si se requiere
    # for label, scores in report.items():
    #     if label not in ["accuracy", "macro avg", "weighted avg"]:
    #         metrics[f'{label}_precision'] = scores['precision']
    #         metrics[f'{label}_recall'] = scores['recall']
    #         metrics[f'{label}_f1'] = scores['f1-score']
    #         metrics[f'{label}_support'] = scores['support']
    
    return metrics

In [8]:
# true_labels = data['n_declaraciones']
# predicted_labels = train_df['predicted_emotion']
# print(len(true_labels))
# print(len(predicted_labels))

# metrics = compute_metrics(true_labels, predicted_labels)
# metrics